### Initialize library

In [1]:
from limp.utils.gen_utils import ltl2dfa, llm4tl, referent_verification, task_structure_verification, get_spatial_referents
from osg.utils.general_utils import load_data
from osg.vlm_library import vlm_library

### Load data

In [ ]:
tmp_fldr=f"results/"

data_format = "r3d"     
data_path = "../data/sample.r3d"   

vlm_instance = vlm_library(vl_model="owl_vit", data_src=data_format, seg_model= "mobile_sam", tmp_fldr=tmp_fldr) 

env_pointcloud, observations_graph = load_data(data_path,                  # path to data
                                               data_format,                # data format [robot: robot collected data / r3d: iphone collected data (via Record3d app)]
                                               tmp_fldr,                   # temp folder to store execution artefacts
                                               pcd_downsample=False,       # flag to downsample generated pointcloud
                                               compression_percentage=60   # reduce rgbd observations used to create map by xx% 
                                               )

### Language to  Ltl

In [ ]:
# Instruction for github: take the bowl next to the xxx to the sink, and bring me, the microwave is acting up so try to not go near it.

In [ ]:
# Go bring the plush toy between the sofa and the television to the couch, my cat is on the brown table so please dont pass near the table when you are returning with the toy

In [ ]:
#Demo instruction: Bring the green plush toy to the whiteboard in front of it

print(f"\n*************************************************************************\nInstruction Following\n*************************************************************************")
input_lang_instruction = input("Enter the natural language description of the task: ")

in_context_examples = "limp/language/temporal_logic/ltl_datasets/efficient-eng-2-ltl-droneplanning"
in_context_count    = 10
lang2ltl_path       = "limp/language/temporal_logic/embedding_cache/small-droneplanning_lang2ltl.pkl" 
lang2embedding_path = "limp/language/temporal_logic/embedding_cache/small-droneplanning_lang2embeddings.pkl"

api_choice = "openai" #googleai / openai
print(f'\nInput instruction: "{input_lang_instruction}"')
print("Running Language Instruction Module ...")

strategy_choice="two_stage_similar_embedding"
# strategy_choice="two_stage_random_embedding"
# strategy_choice="single_stage"

encoding_map, response_ltl, spot_ltl, llm_response_history= llm4tl(api_choice, 
                                                                   input_lang_instruction, 
                                                                   in_context_examples, 
                                                                   lang2embedding_path, 
                                                                   lang2ltl_path, 
                                                                   in_context_count, 
                                                                   enable_prints=False,
                                                                   strategy=strategy_choice)

original_encoding_map, original_response_ltl, original_spot_ltl, original_llm_response_history = encoding_map, response_ltl, spot_ltl, llm_response_history 

print("Spotify predicate encoding map: ", encoding_map)
print("Response LTL formula: ", response_ltl)
print("Cleaned LTL formula: ", spot_ltl,"\n")

display(spot_ltl)

In [ ]:
llm_response_history

### Interactive Symbol Verification

In [ ]:
# Referent verification
encoding_map, response_ltl, spot_ltl, llm_response_history  = referent_verification(input_lang_instruction, encoding_map, response_ltl, spot_ltl, llm_response_history, strategy_choice, api_choice)

In [ ]:
#Task structure verification
encoding_map, response_ltl, spot_ltl, llm_response_history, selected_dfa_path  = task_structure_verification(input_lang_instruction, encoding_map, response_ltl, spot_ltl, llm_response_history, strategy_choice, api_choice)

In [ ]:
# Visualize original and verified results
print("*****************************************\nOriginal Results\n*****************************************")
print("Original encoded formula: ",original_spot_ltl)
print("Original encoding map: ",original_encoding_map)
print("*****************************************\nAfter Verification\n*****************************************")
print("Verified encoded formula: ",spot_ltl)
print("Verified encoding map: ",encoding_map)

### Construct task dfa from ltl

In [ ]:
#constructing task dfa from ltl formula
task_dfa, dfa_graph = ltl2dfa(encoding_map, spot_ltl, visualize_details=True, show_diagram=True, show_labels=True, path=selected_dfa_path)

### Ground referents and filter instances via spatial constraints

In [ ]:
## Extract spatial information
referent_spatial_details = get_spatial_referents(encoding_map)
print("referent_spatial_details: ",referent_spatial_details,"\n")

## Spatial grounding
relevant_element_details = vlm_instance.spatial_grounding(observations_graph, referent_spatial_details, visualize=True, use_segmentation=True, multiprocessing=True, workers=3)

print("Referents after spatial constraint filtering:",len(relevant_element_details))
#for all relevant elements print their ids
print(f"\nFiltered elements \n",[element['mask_id'] for element in relevant_element_details])

##### Select Robot Start Point in Map

In [ ]:
from limp.planner.multi_level_planner import generate_obstacle_map
from limp.utils.fmt_utils import plot_map_with_points
%matplotlib widget

resolution = 0.01
h_min_bottom = -1   #experiment with values to filter out floor
h_max_top = 1       #experiment with values to filter out ceiling
obstacle_map, _, map_min_bound, map_max_bound = generate_obstacle_map(env_pointcloud, None, resolution, h_min_bottom,  h_max_top)

## Visually getting start point from map
clicked_points = plot_map_with_points(obstacle_map)

### Progressive motion planning

In [ ]:
from limp.planner.multi_level_planner import progressive_motion_planner

robot_motion_type            = "2D"                   # determines if planing space is 2D or 3D
z_height_2d                  = 0                      # height of 2D planning space
step_factor                  = 40                     # determines density of generated visual demarcations of regions of interest. Adjust for denser or sparser points
goal_sampling_percentage     = 15                     # percentage of goal points to sample from the goal region to make exhaustive motion planning tractable
use_heuristic_flag           = True                   # determines if we use modified version of FMT* with cost to goal heuristic 
visualize_flag               = True                   # determines if we visualize the computed motion plan
obstacle_map_resolution      = 0.01                   # determines the resolution of the obstacle map
filter_h_min_bottom          = h_min_bottom           # height to filter out pointcloud points below this height (meters) [Floor]      
filter_h_max_top             = h_max_top              # height to filter out pointcloud points above this height (meters) [Roof] #see doors: 0.7 || Old value:-0.15
nearness_threshold           = 0.5                      # determines the meaning of nearness of planning space demarcation (meters)
start_point                  = clicked_points[-1]     # robot start location
show_color_bars_flag         = False

computed_plan = progressive_motion_planner(start_point, 
                                           task_dfa, 
                                           dfa_graph, 
                                           env_pointcloud, 
                                           relevant_element_details, 
                                           encoding_map, 
                                           nearness_threshold, 
                                           obstacle_map_resolution, 
                                           filter_h_min_bottom, 
                                           filter_h_max_top, 
                                           robot_motion_type, 
                                           height_2d=z_height_2d, 
                                           stepfactor=step_factor, 
                                           use_heuristic=use_heuristic_flag,
                                           visualize=visualize_flag,
                                           tmp_fldr=tmp_fldr,
                                           goal_sample_percentage=goal_sampling_percentage,
                                           show_color_bars=show_color_bars_flag)

### View Task and Motion Plan

In [ ]:
computed_plan['world_plan'] 